In [18]:
from collections import defaultdict
import nltk
import os
import numpy as np
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer
import enchant
from gensim import corpora

In [19]:
textos = []
for p, d, f in os.walk(r'machado/machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid), encoding='iso-8859-1') as g:
                textos.append(g.read())

In [20]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [21]:
textos = textos[:4]

In [22]:
def clean_sentence(texto : str):
    return [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
textos = [*map(clean_sentence,textos)]

In [23]:
all_words = set()
for text in textos:
    all_words |= set(text)
ordered_words = list(sorted(all_words))
index_word = dict([(word,i) for i, word in enumerate(ordered_words)])

In [24]:
matrix = [[0]*len(textos) for _ in range(len(all_words))]
for i, document in enumerate(textos):
    for j, word in enumerate(ordered_words):
        matrix[j][i] = document.count(word)
matrix = np.matrix(matrix)
binary_matrix = (matrix >= 1)*1

## Questão 1

In [25]:
def frequency_from(word, matrix):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    
    line = matrix[word_index]
    return line/np.sum(line)

def binary_frequency(word):
    return frequency_from(word,binary_matrix)

def word_frequency(word):
    return frequency_from(word,matrix)

In [26]:
binary_frequency('agora')

matrix([[0.5, 0. , 0. , 0.5]])

In [27]:
word_frequency('agora')

matrix([[0.66666667, 0.        , 0.        , 0.33333333]])

In [28]:
def tfdf(word):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    line = matrix[word_index]
    quantity_words_per_doc = matrix.sum(axis=0)
    tf = line/quantity_words_per_doc
    
    freq_word = ((tf > 0)*1).sum()
    assert freq_word > 0
    df = np.log(matrix.shape[1]/freq_word)
    assert df >= 0
    return tf*df

In [29]:
tfdf('agora')

matrix([[0.00223236, 0.        , 0.        , 0.00063767]])

## Questão 2

In [30]:
def get_log_normalized(matrix):
    log_matrix = np.log(matrix+1)
    squared_log_matrix = np.square(log_matrix)
    sum_rows = squared_log_matrix.sum(axis=1)
    log_normalized_matrix = np.power(np.multiply(squared_log_matrix,np.power(sum_rows, -1)),0.5)
    return log_normalized_matrix

In [31]:
log_normalized_matrix = get_log_normalized(matrix)
log_normalized_matrix

matrix([[0.66824283, 0.74394322, 0.        , 0.        ],
        [0.        , 0.        , 1.        , 0.        ],
        [0.        , 0.70710678, 0.70710678, 0.        ],
        ...,
        [0.        , 0.        , 0.        , 1.        ],
        [0.        , 0.        , 0.        , 1.        ],
        [0.70710678, 0.        , 0.        , 0.70710678]])

## Questão 3

In [44]:
frequency = defaultdict(int)
for text in textos:
    for token in text:
        frequency[token] += 1
freq_texts = [[token for token in text if frequency[token] > 1]
         for text in textos]
dictionary = corpora.Dictionary(freq_texts)

In [45]:
def to_corpus(doc):
    new_vec = dictionary.doc2bow(doc.lower().split())
    corpus = [dictionary.doc2bow(text) for text in freq_texts]
    return corpus

In [50]:
to_corpus('agora')

[[(0, 4),
  (1, 2),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 2),
  (6, 2),
  (7, 2),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 3),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 4),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 2),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 2),
  (36, 2),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 2),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 2),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 2),
  (50, 10),
  (51, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 2),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 2),
  (63, 1),
  (64, 4),
  (65, 1),
  (66, 1),
  (67, 5),
  (68, 1),
  (69, 2),
  (70, 7),
  (71, 2),
  (72, 1),
  (73, 1),
  (74, 3),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 3),
  (81, 2),
  (82, 2),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 3

In [43]:
dictionary